In [1]:
import cvxpy
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import cupy as cp

NUMBER_OF_COUNTIES = 3143

In [2]:
def demand(x, P, totalPop):
    '''
    Calculates demand as a function of price.

    Args:
        x: Total number of stores in given county (cvxpy.Variable).
        P: Price (cvxpy.Variable or cvxpy.Parameter).
        totalPop: Total population in county (cvxpy.Parameter or numeric).

    Returns:
        demand: Demand corresponding to input variables (cvxpy.Expression).
    '''
    a = 0.03 * P  # Coefficient for price
    # L: Maximum demand at unit price 1.0.
    L = cvxpy.multiply(totalPop, 1 / x)  # Equivalent to totalPop / x

    return cvxpy.multiply(L, cvxpy.exp(-cvxpy.multiply(a, P)))

In [3]:
def revenue(P, IR, demand):
    '''
    Calculates Revenue of Product.

    Args:
        P: Price of a given product.
        IR: Income ratio of county to maximum county income (0 to 1).
        demand: The demand of a given product.

    Returns:
        revenue: Demand times Price.
    '''
    sqrt_IR = cvxpy.sqrt(IR)
    # sqrt_IR = np.sqrt(IR)
    answer = []
    for i in range(len(IR.value)):
        answer.append(cvxpy.power(P[i], sqrt_IR[i].value) * demand[i])
    # answer = cvxpy.bmat([cvxpy.power(p, sqrt_ir) * dem for p, sqrt_ir, dem in zip(P.value, sqrt_IR.value, demand.value)])
    # print(answer)
    return cvxpy.hstack(answer)
    # return cvxpy.power(P, sqrt_IR) * demand

In [4]:
def loss(P, IR, demand):
    '''
    Calculates losses: 8.2% of revenue incurred by restaurant.

    Args:
        P: Price of restaurant.
        IR: Income ratio of county to maximum county income (0 to 1).
        demand: Demand of restaurant.

    Returns:
        loss: Losses incurred by restaurant.
    '''
    return 0.082 * revenue(P, IR, demand)

def costs(P, IR, minwage, rent, demand, employees=15):
    '''
    Calculates costs of restaurant.

    Args:
        minwage: Minimum wage in county.
        rent: Rent of restaurant.
        demand: Demand of restaurant.
        P: Price of restaurant.
        IR: Income ratio of county to maximum county income (0 to 1).
        employees: Number of employees in restaurant (default is 15).

    Returns:
        netCosts: Total costs of restaurant.
    '''
    loss_incurred = loss(P, IR, demand)
    operating_costs = cvxpy.multiply(minwage, employees) + rent + loss_incurred
    return operating_costs

In [5]:
def profit(x, P, totalPop, IR, minwage, rent):
    '''
    Calculates Profit = Revenue - Costs.

    Args:
        x: Total number of stores in given county.
        P: Price.
        totalPop: Total population in county.
        IR: Income Ratio relative to maximum county income.
        minwage: Minimum wage in county.
        rent: Rent of restaurant.

    Returns:
        profit: Profit of restaurant.
    '''
    demand_val = demand(x, P, totalPop)
    rev = revenue(P, IR, demand_val)
    cost = costs(P, IR, minwage, rent, demand_val)
    return rev - cvxpy.multiply(x, cost)


In [6]:
def objectiveFunction(x, P, totalPop, IR, minwage, rent):
    '''
    Objective function for optimization problem.

    Args:
        x: Total number of stores + Price per store.
        totalPop: Total population in county.
        IR: Income Ratio relative to maximum county income.
        minwage: Minimum wage in county.
        rent: Rent of restaurant.

    Returns:
        Objective value to minimize.
    '''
    return x.T @ profit(x, P, totalPop, IR, minwage, rent)

In [15]:
def newObjectiveFunction(x, P, totalPop, IR, minwage, rent, employees=15):
    '''
    Objective function for optimization problem.

    Args:
        x: Total number of stores + Price per store.
        totalPop: Total population in county.
        IR: Income Ratio relative to maximum county income.
        minwage: Minimum wage in county.
        rent: Rent of restaurant.

    Returns:
        Objective value to minimize.
    '''
    penalty_factor = 1e10  # A large penalty
    penalty = penalty_factor * cvxpy.sum(cvxpy.pos(10 - P))

    cost_minus_loss = minwage * employees + rent
    
    revenue_x_part1 = cvxpy.multiply(
        totalPop, 
        cvxpy.exp(-cvxpy.multiply(0.03 * P, P))
    ) 
    P_power_sqrt_IR = []
    sqrt_IR = cvxpy.sqrt(IR)
    for i in range(len(IR.value)):
        P_power_sqrt_IR.append(cvxpy.power(P[i], sqrt_IR[i].value))
    P_power_sqrt_IR = cvxpy.hstack(P_power_sqrt_IR)
    revenue_x = cvxpy.multiply(revenue_x_part1, P_power_sqrt_IR)

    formula = 0.918 * revenue_x - cvxpy.multiply(x, cost_minus_loss)
    formula_convex = formula + penalty
    return cvxpy.sum(formula_convex)

In [8]:
def cupy_optimize(budget, N, risk, totalPop, IR, minwage, rent):
    '''
    Optimizes the objective function given certain variables.

    Args:
        totalPop: Total population in county.
        IR: Income Ratio relative to maximum county income.
        budget (int): Total budget for owning restaurants.
        risk (float): Total acceptable risk ratio per location (cost/revenue).
        N (int): Maximum number of stores to open.
        minwage: Minimum wage in county.
        rent: Rent of restaurant.

    Returns:
        result: Result of optimization function.
    '''

    x = cvxpy.Variable(NUMBER_OF_COUNTIES, name='x') # Number of stores in each county
    P = cvxpy.Variable(NUMBER_OF_COUNTIES, name='P') # Price in each county
    budget = cvxpy.Constant(budget, name='budget') # Total budget for owning restaurants
    N = cvxpy.Constant(N, name='N') # Maximum number of stores to open    
    risk = cvxpy.Constant(risk, name='risk') # Total acceptable risk ratio per location (cost/revenue)
    totalPop = cvxpy.Constant(totalPop, name='totalPop') # Total population in county
    IR = cvxpy.Constant(IR, name='IR') # Income Ratio relative to maximum county income
    minwage = cvxpy.Constant(minwage, name='minwage') # Minimum wage in county
    rent = cvxpy.Constant(rent, name='rent') # Rent of restaurant


    # ######################################################
    # Inequality constraints
    # ######################################################

    constraints = []
    demand_val = demand(x, P, totalPop)
    cost = costs(P,IR,minwage,rent,demand_val)
    total_cost = x.T @ cost # Get total costs for budget constraint
    rev = revenue(P, IR, demand_val) # Get revenue for risk constraint

    constraints.append(budget - total_cost >= 0) # Budget constraint
    constraints.append(N - sum(x) >= 0) # Maximum number of stores constraint
    constraints.append(risk - cost/rev >= 0) # Risk constraint
    constraints.append(x >= 0) # Number of stores must be positive
    constraints.append(P >= 10) # Price must be positive
    constraints.append(P <= 50) # Price must be positive

    objective = newObjectiveFunction(x, P, totalPop, IR, minwage, rent) # Get objective function
    problem = cvxpy.Problem(
        cvxpy.Maximize(objective), 
        constraints
    ).solve() # Solve the problem
    result = {
        'x': x.value, # Number of stores in each county
        'P': P.value, # Price in each county
    }
    return result


In [9]:
budget = 10000000
N = 4000
risk = 0.8

print(budget, N, risk)

from readData import *
from rent_estimation import calculate_rent_estimation

income = getIncome()
income.drop(['MedianIncome','MedianIncomeRatio'],axis=1,inplace=True)
populations = getPopulation()
populations = populations[['State','County','2024']]
populations.rename({'2024':'Population'},axis=1,inplace=True)
rent = calculate_rent_estimation()
rent = rent[['State','County','Estimated_annual_rent']]
minwage = getMinWage()

data = populations.merge(income)
data = data.merge(rent)
data = data.merge(minwage)
data

10000000 4000 0.8
Average Rent Per Sqft: 92.93
Average Annual Rent: 185857.18


c:\MY FILES\UMD\Sem 4\MSML604 Optimization\Project\optimization-location-intelligence\rent_estimation.py:110: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  merged_df['Commercial_rent_per_sqft_year'].fillna(avg_rent_per_sqft, inplace=True)
c:\MY FILES\UMD\Sem 4\MSML604 Optimization\Project\optimization-location-intelligence\rent_estimation.py:111: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace 

,State,County,Population,CountyID,MeanIncome,MeanIncomeRatio,Estimated_annual_rent,MinWage
0,Alabama,Autauga,61464,0500000US01001,93367,0.411925,184682.27,7.25
1,Alabama,Baldwin,261608,0500000US01003,100105,0.441653,282247.68,7.25
2,Alabama,Barbour,24358,0500000US01005,64745,0.285648,106381.32,7.25
3,Alabama,Bibb,22258,0500000US01007,67735,0.298840,146669.94,7.25
4,Alabama,Blount,60163,0500000US01009,79203,0.349435,168697.13,7.25
...,...,...,...,...,...,...,...,...
3138,Wyoming,Sweetwater,41273,0500000US56037,98608,0.435048,211876.05,7.25
3139,Wyoming,Teton,23272,0500000US56039,185173,0.816964,1621554.10,7.25
3140,Wyoming,Uinta,20621,0500000US56041,89995,0.397048,235265.57,7.25
3141,Wyoming,Washakie,7662,0500000US56043,80444,0.354910,184506.77,7.25


In [16]:
results = cupy_optimize(budget = budget,
                   N = N,
                   risk = risk,
                   totalPop = data['Population'],
                   IR = data['MeanIncomeRatio'],
                   minwage = data['MinWage'],
                   rent = data['Estimated_annual_rent'])


c:\MY FILES\UMD\Sem 4\MSML604 Optimization\.venv\Lib\site-packages\cvxpy\problems\problem.py:158: UserWarning: Objective contains too many subexpressions. Consider vectorizing your CVXPY code to speed up compilation.
  warnings.warn("Objective contains too many subexpressions. "
c:\MY FILES\UMD\Sem 4\MSML604 Optimization\.venv\Lib\site-packages\cvxpy\problems\problem.py:164: UserWarning: Constraint #0 contains too many subexpressions. Consider vectorizing your CVXPY code to speed up compilation.
  warnings.warn(f"Constraint #{i} contains too many subexpressions. "
c:\MY FILES\UMD\Sem 4\MSML604 Optimization\.venv\Lib\site-packages\cvxpy\problems\problem.py:164: UserWarning: Constraint #2 contains too many subexpressions. Consider vectorizing your CVXPY code to speed up compilation.
  warnings.warn(f"Constraint #{i} contains too many subexpressions. "


DCPError: Problem does not follow DCP rules. Specifically:
The objective is not DCP. Its following subexpressions are not:
Promote(0.03, (3143,)) @ P @ P